In [1]:
import rasa.utils.io as io_utils
io_utils.write_yaml_file(
    {
        "language": "en",
        "pipeline": "supervised_embeddings",
        "policies": [{"name": "KerasPolicy"}],
    },
    "./out/config_1.yml",
)

In [19]:
cnt=io_utils.read_yaml_file('./templates/domain.yml')
intents=cnt['intents']
# intents
intents.append({'new_act':{'triggers': 'action_is_bot'}})
intents

['greet',
 'goodbye',
 'affirm',
 'deny',
 'mood_great',
 'mood_unhappy',
 'bot_challenge',
 {'ask_is_bot': {'triggers': 'action_is_bot'}},
 {'new_act': {'triggers': 'action_is_bot'}}]

In [20]:
cnt['actions']

['utter_greet',
 'utter_cheer_up',
 'utter_did_that_help',
 'utter_happy',
 'utter_goodbye',
 'utter_iamabot',
 'action_hello_world',
 'action_is_bot']

In [10]:
io_utils.write_yaml_file(cnt, "./out/domain_1.yml")

In [14]:
import glob
glob.glob('/pi/stack/conf/ruleset_*.json')

['/pi/stack/conf/ruleset_basic.json']

In [21]:
cnt=io_utils.read_yaml_file('./templates/domain.yml')
intents=cnt['intents']
actions=cnt['actions']
for f in glob.glob('/pi/stack/conf/ruleset_*.json'):
    rules=io_utils.read_json_file(f)
    for rule in rules:
        intents.append({rule['intent']:{'triggers': rule['action']}})
        actions.append(rule['action'])

io_utils.write_yaml_file(cnt, "./out/domain_1.yml")
print('done')

done


In [24]:
from rasa.train import train_async

from rasa.utils.endpoints import ClientResponseError, EndpointConfig
from rasa.core.agent import Agent
from rasa.core.interpreter import RasaNLUInterpreter
from rasa.model import get_model, get_latest_model
from rasa.core.channels.channel import UserMessage

# preq: $ start actions

bot='genesis'

endpoint = EndpointConfig("http://localhost:5055/webhook")
bot_locs={'genesis': '/pi/ws/sagas-ai/bots/genesis'}

prefix=f"{bot_locs[bot]}"
# domain_file="./out/domain_1.yml"
domain_file=f"{prefix}/domain.yml"
io_utils.write_yaml_file(cnt, domain_file)
await train_async(
        domain=domain_file,
        config=f"{prefix}/config.yml",
        training_files=f"{prefix}/data/",
        output_path=f"{prefix}/models/",
    )

Processed Story Blocks:   0%|          | 0/5 [00:00<?, ?it/s, # trackers=20]

Training Core model...


Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 46.97it/s, # actions=16]
Processed actions: 16it [00:00, 368.83it/s, # examples=16]


Core model training completed.
NLU data/configuration did not change. No need to retrain NLU model.
Your Rasa model is trained and saved at '/pi/ws/sagas-ai/bots/genesis/models/20191206-011925.tar.gz'.


'/pi/ws/sagas-ai/bots/genesis/models/20191206-011925.tar.gz'

In [27]:
bot_loc=get_latest_model(f"{bot_locs[bot]}/models")
print(f'.. load bot model {bot_loc}')
agent = Agent.load(bot_loc, action_endpoint=endpoint)

# await agent.handle_text("do you have any restaurants")
# await agent.handle_text("hello")
text = '/behave_purpose{"object_type": "restaurant", "sents":"%s"}'%"do you have any restaurants"
message = UserMessage(text, sender_id="my")
result = await agent.handle_message(message)
result

.. load bot model /pi/ws/sagas-ai/bots/genesis/models/20191206-011925.tar.gz
INFO:tensorflow:Restoring parameters from /var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpjfv_oq2w/nlu/component_6_EmbeddingIntentClassifier.ckpt


[{'recipient_id': 'my', 'custom': {'result': 'log ok'}}]